<a href="https://colab.research.google.com/github/devyulbae/AIClass/blob/main/ex_w1d3_logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

In [ ]:
# colab import
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 문자열을 건너뛰고 읽기
xy = np.genfromtxt('/content/drive/MyDrive/datas/logistic_regression_dataset_16_features.csv',
                   delimiter=',', dtype=None, encoding=None)

x_train = xy[1:-100, 0:-1]
y_train = xy[1:-100, [-1]]
x_train, y_train = x_train.astype(np.float32), y_train.astype(np.float32)

x_test = xy[-100:, 0:-1]
y_test = xy[-100:, [-1]]
x_test, y_test = x_test.astype(np.float32), y_test.astype(np.float32)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print("x_test: \n", x_test[0])
print("y_test: \n", y_test[0])

In [ ]:
# data slicing
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

In [ ]:
W = tf.Variable(tf.random.normal([16,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

In [ ]:
# functions
def logistic_regression(features):
    hypothesis  = tf.divide(1., 1. + tf.exp(-(tf.matmul(features, W) + b)))
    return hypothesis

def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean( labels*tf.math.log(hypothesis)+ (1-labels)*tf.math.log(1-hypothesis))
    return cost

def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

In [ ]:
# optimizer
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=1e-2)

In [ ]:
# main
epochs = 10001

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      pred = logistic_regression(features)
      loss_value = loss_fn( pred, labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 500 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),labels)))

In [ ]:
# test
test_acc = accuracy_fn(logistic_regression(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))